In [0]:
from delta.tables import *

DeltaTable.create(spark)\
    .tableName("dimemployee")\
    .addColumn("empid","INT")\
    .addColumn("empName","STRING")\
    .addColumn("SSN","STRING")\
    .execute()

In [0]:
%sql
insert into dimemployee values(100,"Mike",'2345776');
insert into dimemployee values(200,"Davis",'5632178');
insert into dimemployee values(300,"Peter",'1456782');

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
select * from dimemployee

empid,empName,SSN
200,Davis,5632178
300,Peter,1456782
100,Mike,2345776


In [0]:
pip install cryptography

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython() 

In [0]:
from cryptography.fernet import Fernet
key=Fernet.generate_key()
f=Fernet(key)

In [0]:
PIIData=b"pradeep484@gmail.com"
test_data=f.encrypt(PIIData)
print(test_data)

b'gAAAAABnv_Zs0L4j7wy1yoFd6Je7G_XT_RYwKAVCpIXjWeAxwfLoqjOjCKZFqtAR1liK00gwrPuTeXnoM705MyWGi8VRC4jCAuFOjHgEF731JTSdKRP6dMg='


In [0]:
print(f.decrypt(test_data))

b'pradeep484@gmail.com'


In [0]:
def encrypt_data(data,KEY):
    from cryptography.fernet import Fernet
    f=Fernet(KEY)
    dataB=bytes(data,'utf-8')
    encrypted_data=f.encrypt(dataB)
    encrypted_data=str(encrypted_data.decode('ascii'))
    return encrypted_data

In [0]:
def decrpt_data(encrypted_data,KEY):
    from cryptography.fernet import Fernet
    f=Fernet(KEY)
    decrypted_data=f.decrypt(encrypted_data.encode()).decode()
    return decrypted_data

In [0]:
from pyspark.sql.functions import lit,udf,md5
from pyspark.sql.types import StringType
encryption = udf(encrypt_data,StringType())
decryption = udf(decrpt_data,StringType())

In [0]:
df=spark.table("dimemployee")
encryptDf=df.select ("*",
                     (encryption("SSN",lit(key))).alias("encrypted_SSN"))
encryptDf.display()

empid,empName,SSN,encrypted_SSN
200,Davis,5632178,gAAAAABnv_4ko-rWmQDN90hU9cgAdR58qOoMZVRKKyDwlOHUFoizDmrY4Ui2ZmgHLOxTuOe9Ftp9a3cs3uRqOA2aOSc8NjK3Qw==
300,Peter,1456782,gAAAAABnv_4lsz3GcEMb1Gp0jgLZymjklBgglF52rAu-k3HmVPXOMraukETM2qLRlIJyC5jlQ-ge3YFPu3IiJ27jNr5A8RPZEg==
100,Mike,2345776,gAAAAABnv_4lKm__uSBVh_Ebw19hH2vBbhi4RW9ZKMRY-QxagLIjswDxDxdM4DyyLXckCeKMMc0Pqbr-SZvHKxaaAxUaELlBxQ==


In [0]:
decryptDf=encryptDf.select ("*",
                            (decryption("encrypted_SSN",lit(key))).alias("decrypted_SSN"))
decryptDf.display()

empid,empName,SSN,encrypted_SSN,decrypted_SSN
200,Davis,5632178,gAAAAABnv_6jqHepdPuduR5PEDwHVj8NIEIR8piUnHtVsBbbR7qCRWqRuzotr1G5sbDSwH2T0ODkvNusZ9pdNBcS5nePhH7U7A==,5632178
300,Peter,1456782,gAAAAABnv_6jY3QjHJvfvIwuChqF5u3bn7xPlNQUSV9k2wocjW-XPMfyHt3zXaS1_fubXDo1VuguBY5yZzmg8SC0Dx6pe-9qOw==,1456782
100,Mike,2345776,gAAAAABnv_6jfkpDtyMAjfHVt8HUWTE5kpRa7Jrc9qLRx-mHq-ssB_YVDQwWkADt4zZZPvosXAdkKSDY_qNEKhAOn8HcLJm5Ug==,2345776
